In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def find_words_with_excluded_ingredients(food_data, excluded_ingredients):
    except_list = []
    for ingredients in food_data['RecipeIngredientParts']:
        words = ingredients.split()  # 각 행의 단어들을 분리
        for word in words:
            for ingredient in excluded_ingredients:
                if ingredient.lower() in word.lower():  # 단어에 'excluded_ingredients'에 있는 재료가 포함되어 있는지 확인 (대소문자 구분 없이)
                    except_list.append(word)
    return except_list

def filter_food_data(food_data, except_list, calories_threshold):
    # 새로운 DataFrame을 생성하여 필터링된 결과를 저장
    filtered_food_data = food_data.copy()
    
    # 사용자가 제외하고자 하는 음식 재료들을 필터링
    for ingredient in except_list:
        filtered_food_data = filtered_food_data[filtered_food_data['RecipeIngredientParts'] != ingredient]
    
    # 사용자의 하루 권장 칼로리보다 적은 음식들을 필터링
    filtered_food_data = filtered_food_data[filtered_food_data['Calories'] < calories_threshold]
    
    return filtered_food_data

def get_recommendations(user_keywords, tfidf_vectorizer, tfidf_matrix, food_data, user_calories_threshold):
    keywords = user_keywords
    user_tfidf_matrix = tfidf_vectorizer.transform([' '.join(keywords)])
    cosine_sim = cosine_similarity(user_tfidf_matrix, tfidf_matrix)
    sim_scores = list(cosine_sim[0])  # 유사도 값만 리스트에 담기
    return sim_scores

def user_required_calories(user_height, user_weight, user_age, user_gender):
    height_m = user_height / 100
    weight_kg = user_weight
    bmi = weight_kg / (height_m ** 2)

    if user_gender == 'Male':
        bmr = 88.362 + (13.397 * weight_kg) + (4.799 * height_m * 100) - (5.677 * user_age)
    else:
        bmr = 447.593 + (9.247 * weight_kg) + (3.098 * height_m * 100) - (4.330 * user_age)

    calories_threshold = bmr * 1.2
    return calories_threshold

def get_userCategoryList(user1_category):
    userCategoryList = []
    for keyword in user1_category:
        if keyword == 'Healthy':
            userCategoryList.append(['vegetable', 'tomatoes', 'potatoes', 'dry', 'beans', 'beef',\
                                   'chicken', 'pork', 'nonfat', 'lowfat', 'beets', 'apples', 'fish',\
                                   'cheese', 'light', 'regular', 'dressing', 'juice', 'yogurt', 'olive',\
                                   'steak', 'meat'])
        if keyword == 'Diet':
            userCategoryList.append(['lemon', 'cabbage', 'tomatoes', 'potatoes', 'dressing', 'light',\
                                    'regular', 'juice', 'yogurt', 'eggs', 'salmon', 'lowfat', 'nonfat',\
                                    'dried', 'mushroom' ,'garlic', 'chicken'])
        return userCategoryList 

def category_choice_recommend(user_categorical_list, similarities, tfidf_vectorizer, tfidf_matrix, food_data, user_calories_threshold):
    # 유저가 선택한 카테고리의 데이터(user_categorical_list)를 받아서 keyword에 리스트 형태로 저장
    combined_keywords = [' '.join([keyword for keyword in category_keywords]) for category_keywords in user_categorical_list]
   
    # 유저가 선택한 카테고리의 데이터를 TF-IDF행렬로 변환
    user_tfidf_matrix = tfidf_vectorizer.transform(combined_keywords)

    # 유저와 음식간의 키워드 유사도 값을 matrix로 사용
    food_similarity_matrix = [float(similarity) for similarity in similarities]

    # 코사인 유사도를 통하여 유저가 선택한 카테고리와 음식간의 유사도를 구함
    user_category_similarity = cosine_similarity(user_tfidf_matrix, tfidf_matrix)

    # (유저와 음식간의 유사도)와 (유저가 선택한 카테고리와 음식간의 유사도)를 구해서 평균으로 만듬
    final_cosine_sim = (user_category_similarity + food_similarity_matrix) / 2

    # 위 코사인 유사도 행렬에서 첫 번째 행(사용자 입력과 음식 간의 유사도 값)을 가져옴
    sim_scores = list(enumerate(final_cosine_sim[0]))

    # 유사도를 기준으로 리스트를 내림차순으로 정렬
    # key=lambda x: x[1]는 리스트의 각 요소를 유사도에 대해 정렬하기 위한 키 함수
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # 가장 높은 유사도(쏠림값)을 제외하고 전부 반환
    sim_scores = sim_scores[1:]

    # 각 유사도의 index 조회
    food_indices = [idx[0] for idx in sim_scores]

    # 조회된 index값으로 food데이터셋에서 'Name', 'Calories'조회
    recommended_food = food_data.iloc[food_indices][['Name', 'Calories']]

    # 추천된 값을 user_calories_threshold기준으로 필터링
    # 음식 데이터 전체와 추천 데이터의 코사인 유사도를 이용하여 food데이터셋에서 추천 데이터가 다시 나왔으므로
    # 기존 추천에서 걸러졌던 user_calories_threshold 필터링 과정을 다시 수행
    recommended_food = recommended_food[recommended_food['Calories'] < user_calories_threshold]
    return recommended_food

def recommend_similar_food(similarities, food, user_calories_threshold):
    recommended_foods = []
    for name, calories in similarities.items():
        # 음식과 사용자의 선호도 간의 코사인 유사도 계산
        # 유사도가 가장 높은 상위 30개 인덱스 추출
        recommended_food_indices = np.argsort(calories)[:30][::-1]
        # 추천된 음식들의 이름과 인덱스, 칼로리 저장
        recommended_foods.extend([(idx, food.iloc[idx]['Name'], food.iloc[idx]['Calories']) for idx in recommended_food_indices])
    # DataFrame으로 변환
    recommended_foods_df = pd.DataFrame(recommended_foods, columns=['Index', 'Name', 'Calories'])
    # 사용자의 칼로리 요구량보다 낮은 음식만 필터링
    recommended_foods_df = recommended_foods_df[recommended_foods_df['Calories'] < user_calories_threshold]
    return recommended_foods_df

# 메인 실행()
# 음식 데이터 로드
food = pd.read_csv('./foodData4_24.csv', dtype={'Calories': float})
food['Keywords'] = food['Keywords'].fillna('')

# 사용자1의 예제 데이터
user1 = ['Dessert', 'Pork', 'Grains', 'Oven', 'Christmas', 'Brunch']
user1_category = ['Healthy', 'Diet']
user_calories_threshold = user_required_calories(178.9, 72.3, 24, 'Male') / 3

# 유저가 카테고리를 선택하면 해당 카테고리에서 유저에게 알맞은 형태의 '재료'값을 리스트 형으로 반환
user_categorical_list = get_userCategoryList(user1_category)

# 음식 데이터셋 필터링
# 음식 데이터에서 제외할 재료 입력받음
excluded_ingredients = ['egg', 'water', 'olive oil', 'butter', 'onion', 'honey', 'garlic', 'banana'] 
except_list = find_words_with_excluded_ingredients(food, excluded_ingredients)
unique_except_list = list(set(except_list))

# 데이터 필터링 함수 호출
food = filter_food_data(food, unique_except_list, user_calories_threshold)

# TF-IDF 행렬 생성
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(food['Keywords']) # food['Keywords']값을 기반으로 TF-IDF행렬로 변환
tfidf_matrix2 = tfidf.transform(food['RecipeIngredientParts'])  # 기존에 학습된 tfidf 객체를 사용하여 TF-IDF행렬로 변환

# 유저의 키워드와 food데이터셋의 키워드를 코사인 유사도를 이용하여 높은 유사도 순으로 반환
# 반화되는 값 sim_scores :유사도 점수
similarities = get_recommendations(user1, tfidf, tfidf_matrix, food, user_calories_threshold)
# 데이터 추천값을 키워드 기반 유사도에서 카테고리 선택을 기반으로한 추천값으로 변경
similarities = category_choice_recommend(user_categorical_list, similarities, tfidf, tfidf_matrix, food, user_calories_threshold)
# 위 추천값을 영양소 기반으로한 추천값으로 변경
recommendations = recommend_similar_food(similarities, food, user_calories_threshold)

print("최종 추천 음식:")
print(recommendations)

최종 추천 음식:
     Index                                               Name  Calories
0    86919                             Calabrian Tomato Salad     149.7
1    82771                          Pomegranate Pajamba Juice     125.5
2    79492                                Chocolate Milkshake     447.4
3    86346                                    Chicken Lasagna     365.0
4    81368                          Wicked Pink Lady Lemonade     195.8
5     4463                   Apple Raisin Sauce for Baked Ham      52.2
6    68725              My Michelle's Chicken Alfredo Lasagne      29.5
7    83027                           Parmesan Chicken Breasts     499.3
8    12891                   Slow-Cooked Sausage Spanish Rice     347.9
9    31493                              Split Pea Soup (oamc)     212.9
10   13960                          Peanut Brittle in a Jiffy     109.5
11   43717  Roasted Root Vegetables With Maple Balsamic Dr...     240.2
12   68255                     Peppery Greens With Wat